In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import multiprocessing as mp
from multiprocessing import freeze_support

"""
baseline 모델을 설계 및 학습하기 위한 준비
"""

is_cuda = torch.cuda.is_available()
DEVICE = torch.device('cuda' if is_cuda else 'cpu')

print('Current cuda device is', DEVICE)

Batch_Size = 256
Epoch = 30

transform_base = transforms.Compose([transforms.Resize((64,64)),
                                                       transforms.ToTensor()])

train_dataset = ImageFolder(root='./tomato_2/train', transform=transform_base)

val_dataset = ImageFolder(root='./tomato_2/val', transform=transform_base)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=Batch_Size, shuffle=True,
                                           num_workers=4)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                           batch_size=Batch_Size, shuffle=True,
                                           num_workers=4)

print('train_dataset_size: ', len(train_dataset))
print('validation_dataset_size: ', len(val_dataset))

Current cuda device is cpu
train_dataset_size:  6000
validation_dataset_size:  2000


In [2]:
import os
train_dir = './tomato_2/train'
train_classes_list = os.listdir(train_dir)
print('train_lists ', train_classes_list)

train_lists  ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']


In [3]:
"""
baseline 모델 설계
"""
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool =  nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE) #정의한 cnn모댈 Net()의 새로운 객체를 생성 후 to(DEVICE)를 통해 모델을 현재 사용중인 장비에 할당함
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [4]:
"""
   모델 학습을 위한 함수
"""
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        

In [5]:
"""
   모델 평가를 위한 함수
"""
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss = F.cross_entropy(output,
                        target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy #측정한 정확도와 손실을 반환

In [6]:
"""
    모델 학습 실행하기
"""
import time
import copy
import os

def train_baseline(model, train_loader, 
                   val_loader, optimizer, num_epoch = 30):
    
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epoch + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('------------epoch{}-------------'.format(epoch))
        print('train loss: {: .4f}, accuracy: {: .2f}%'.format(train_loss, train_acc))
        print('val loss: {: .4f}, accuracy: {: .2f}%'.format(val_loss, val_acc))
        print('Completed in {: .0f}m {: .0f}s'.format(time_elapsed//60, time_elapsed%60))
        
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, 
                      val_loader, optimizer, Epoch)

torch.save(base, 'baseline.pt')

------------epoch1-------------
train loss:  0.0440, accuracy:  19.52%
val loss:  0.2454, accuracy:  19.35%
Completed in  1m  4s
------------epoch2-------------
train loss:  0.0400, accuracy:  34.72%
val loss:  0.2210, accuracy:  34.35%
Completed in  1m  5s
------------epoch3-------------
train loss:  0.0284, accuracy:  49.83%
val loss:  0.1611, accuracy:  47.70%
Completed in  1m  10s
------------epoch4-------------
train loss:  0.0247, accuracy:  55.98%
val loss:  0.1322, accuracy:  55.40%
Completed in  1m  16s
------------epoch5-------------
train loss:  0.0204, accuracy:  62.87%
val loss:  0.1171, accuracy:  61.60%
Completed in  1m  10s
------------epoch6-------------
train loss:  0.0188, accuracy:  68.28%
val loss:  0.1051, accuracy:  67.05%
Completed in  1m  7s
------------epoch7-------------
train loss:  0.0146, accuracy:  73.43%
val loss:  0.0860, accuracy:  70.25%
Completed in  1m  1s
------------epoch8-------------
train loss:  0.0126, accuracy:  76.12%
val loss:  0.0780, accu

In [7]:
"""
   transfer learning을 위한 준비
"""
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                              [0.229, 0.224, 0.225])
    ]), 
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                              [0.229, 0.224, 0.225])
    ])
}

data_dir = './tomato_2'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), 
            transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
            batch_size=Batch_Size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [8]:
print('train_dataset_size: ',len(image_datasets['train']))
print('val_dataset_size: ',len(image_datasets['val']))

train_dataset_size:  6000
val_dataset_size:  2000


In [9]:
"""
   pre-trained model 불러오기
"""
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, 
                        resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,
                                       step_size=7, gamma=0.1)

c:\Users\smdev\anaconda3\envs\my_pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\smdev\anaconda3\envs\my_pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
"""
   pre_trained model의 일부 layer freeze하기
"""
ct = 0
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [11]:
"""
   transfer learning 모델 학습과 검증을 위한 함수
"""
def train_resnet(model, criterion, optimizer, scheduler, num_epoch = 25):
    
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(num_epoch):
        print('------------epoch{}-------------'.format(epoch + 1))
        since = time.time()
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate: ', l_r)
                
            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]
            
            print('{} loss: {: .4f}, acc: {: .4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('Completed in {: .0f}m {: .0f}s'.format(time_elapsed//60, time_elapsed%60))
        
    print('Best val acc: {: 4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    
    return model    

In [12]:
"""
   모델 학습 실행하기
"""
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, 
                      exp_lr_scheduler, num_epoch=Epoch)

torch.save(model_resnet50, 'resnet_50.pt')

------------epoch1-------------
learning rate:  [0.001]
train loss:  0.8137, acc:  0.7398
val loss:  1.5613, acc:  0.7500
Completed in  3m  9s
------------epoch2-------------
learning rate:  [0.001]
train loss:  0.2787, acc:  0.9057
val loss:  0.3615, acc:  0.8850
Completed in  3m  29s
------------epoch3-------------
learning rate:  [0.001]
train loss:  0.2060, acc:  0.9313
val loss:  0.4612, acc:  0.8610
Completed in  3m  36s
------------epoch4-------------
learning rate:  [0.001]
train loss:  0.1589, acc:  0.9462
val loss:  0.2290, acc:  0.9320
Completed in  3m  45s
------------epoch5-------------
learning rate:  [0.001]
train loss:  0.1392, acc:  0.9507
val loss:  0.2081, acc:  0.9415
Completed in  2m  11s
------------epoch6-------------
learning rate:  [0.001]
train loss:  0.1235, acc:  0.9593
val loss:  0.2040, acc:  0.9375
Completed in  2m  10s
------------epoch7-------------
learning rate:  [0.0001]
train loss:  0.1193, acc:  0.9610
val loss:  0.1635, acc:  0.9490
Completed in  

In [13]:
"""
모델 평가    
"""
transform_base = transforms.Compose([transforms.Resize([64,64]),
                                                       transforms.ToTensor()])

test_base = ImageFolder(root='./tomato_2/test', transform=transform_base)

test_loader = torch.utils.data.DataLoader(test_base,
                                           batch_size=Batch_Size, shuffle=True,
                                           num_workers=4)

In [14]:
"""
Transfer Learning 모델 평가를 위한 전처리    
"""
transforms_resNet = transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
test_resNet = ImageFolder(root='./tomato_2/test', transform=transforms_resNet)
    
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,
                                           batch_size=Batch_Size, shuffle=True,
                                           num_workers=4)

In [16]:
"""
Baseline 모델 성능 평가    
"""
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader)

print('baseline test acc: ', test_accuracy)

"""
Transfer Learning 모델 성능 평가
"""
resnet50 = torch.load('resnet_50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc: ', test_accuracy)

baseline test acc:  89.75
ResNet test acc:  98.1
